In [1]:
### Get Quality Metrics

# Import libraries
import os
import pandas as pd
import numpy as np

ImportError: Unable to import required dependencies:
numpy: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [ ]:
# want to read a specific quality url
url ='https://obsic-metrics.whoi.edu/static/images/quality/2P/OBS04_EDH.txt'

# read in the data
data = pd.read_csv(url, delimiter=',', header=2, names=['DateTime', 'Quality'])

# now specify start and end window

start_date = '2023-10-01 00:00:00'
end_date = '2023-10-31 23:59:55'
mask = (data['DateTime'] >= start_date) & (data['DateTime'] <= end_date)
filtered_data = data.loc[mask]

In [ ]:
data

,DateTime,Quality
0,2024-11-27,111111111111111111111111
1,2024-11-28,111111111111111111101111
2,2024-11-29,111111111111111111111111
3,2024-11-30,111111111111111111111111
4,2024-12-01,111111111111111111111111
...,...,...
113,2025-03-20,000000000000000000000000
114,2025-03-21,000000000000000000000000
115,2025-03-22,000000000000000000000000
116,2025-03-23,000000000000000000000000
